In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import nltk
from nltk.stem import WordNetLemmatizer 

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [47]:
df = pd.read_csv("blogtext.csv")
print(df.shape)
df.head()

(681284, 7)


,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


In [48]:
df.drop(df[df['topic']=='Student'].index,inplace=True)

In [49]:
df.drop(df[df['text'].map(len)<50].index,inplace=True)

In [51]:
df.drop(['gender','age','sign','date'],axis=1,inplace=True)

In [52]:
df.reset_index()


,index,id,topic,text
0,4,3581210,InvestmentBanking,Thanks to Yahoo!'s Toolbar I can ...
1,5,3581210,InvestmentBanking,I had an interesting conversation...
2,6,3581210,InvestmentBanking,Somehow Coca-Cola has a way of su...
3,7,3581210,InvestmentBanking,"If anything, Korea is a country o..."
4,8,3581210,InvestmentBanking,Take a read of this news article ...
5,9,3581210,InvestmentBanking,I surf the English news sites a l...
6,10,3581210,InvestmentBanking,"Ah, the Korean language...it look..."
7,11,3581210,InvestmentBanking,If you click on my profile you'll...
8,12,3581210,InvestmentBanking,Last night was pretty fun...mostl...
9,13,3581210,InvestmentBanking,There is so much that is differen...


In [62]:
d = dict()
for a in df['topic']:
    if a in d:
        d[a]+=1
    else:
        d[a]=1
df.drop(df[df['topic']=='indUnk'].index,inplace=True)
df.describe()

,id
count,2.670570e+05
mean,2.377485e+06
std,1.248855e+06
min,7.596000e+03
25%,1.188176e+06
50%,2.621121e+06
75%,3.512293e+06
max,4.336871e+06


In [64]:
df.reset_index()

,index,id,topic,text
0,4,3581210,InvestmentBanking,Thanks to Yahoo!'s Toolbar I can ...
1,5,3581210,InvestmentBanking,I had an interesting conversation...
2,6,3581210,InvestmentBanking,Somehow Coca-Cola has a way of su...
3,7,3581210,InvestmentBanking,"If anything, Korea is a country o..."
4,8,3581210,InvestmentBanking,Take a read of this news article ...
5,9,3581210,InvestmentBanking,I surf the English news sites a l...
6,10,3581210,InvestmentBanking,"Ah, the Korean language...it look..."
7,11,3581210,InvestmentBanking,If you click on my profile you'll...
8,12,3581210,InvestmentBanking,Last night was pretty fun...mostl...
9,13,3581210,InvestmentBanking,There is so much that is differen...


In [65]:
def preprocess(text):
    return [w for w in gensim.utils.simple_preprocess(text) if w not in gensim.parsing.preprocessing.STOPWORDS and len(w)>3]
def lemmatize(text):
    return [WordNetLemmatizer().lemmatize(w) for w in text]

In [66]:
preproc_doc = []
for s in df['text']:
    preproc_doc.append(lemmatize(preprocess(s)))
    

In [67]:
dwords = gensim.corpora.Dictionary(preproc_doc)

In [68]:
c = 0
for k,v in dwords.iteritems():
    c+=1
    print(k,v)
    if c==10:
        break

0 audio
1 button
2 capture
3 choose
4 click
5 cool
6 enjoy
7 hour
8 instruction
9 korean


In [69]:
dwords.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)


In [70]:
bow = [dwords.doc2bow(s) for s in preproc_doc]

In [73]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow)
corpus_tfidf = tfidf[bow]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5736997760936114),
 (1, 0.15724659945370864),
 (2, 0.1866402276540447),
 (3, 0.13914152151545042),
 (4, 0.13832689677211982),
 (5, 0.09042247351906292),
 (6, 0.11323731293037777),
 (7, 0.0769655743088099),
 (8, 0.18183009714202764),
 (9, 0.1850448032663647),
 (10, 0.033268858607848346),
 (11, 0.2175777013157439),
 (12, 0.07382144018686897),
 (13, 0.061726923248351155),
 (14, 0.28977120425655534),
 (15, 0.18312180741814743),
 (16, 0.10016692517772093),
 (17, 0.22647528497158337),
 (18, 0.10640736864076424),
 (19, 0.24500636690853603),
 (20, 0.26865962225161516),
 (21, 0.07644083792853062),
 (22, 0.2583159530088372),
 (23, 0.16110664271570171)]


In [74]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=30, id2word=dwords, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.004*"went" + 0.003*"today" + 0.003*"urllink" + 0.003*"haha" + 0.003*"like" + 0.003*"night" + 0.003*"gonna" + 0.003*"time" + 0.002*"perth" + 0.002*"going"
Topic: 1 Word: 0.013*"urllink" + 0.011*"test" + 0.009*"christmas" + 0.007*"level" + 0.006*"belle" + 0.005*"moderate" + 0.005*"merry" + 0.005*"personality" + 0.005*"violent" + 0.005*"blogsite"
Topic: 2 Word: 0.003*"like" + 0.003*"time" + 0.002*"night" + 0.002*"little" + 0.002*"went" + 0.002*"thing" + 0.002*"good" + 0.002*"know" + 0.002*"think" + 0.002*"going"
Topic: 3 Word: 0.017*"urllink" + 0.017*"quiz" + 0.014*"harry" + 0.012*"ross" + 0.012*"potter" + 0.006*"pirate" + 0.006*"poll" + 0.005*"portal" + 0.005*"olivia" + 0.005*"rated"
Topic: 4 Word: 0.007*"diet" + 0.006*"weight" + 0.006*"urllink" + 0.006*"pound" + 0.005*"tucker" + 0.005*"sidebar" + 0.004*"mar" + 0.003*"monty" + 0.003*"maureen" + 0.003*"python"
Topic: 5 Word: 0.008*"cheese" + 0.008*"food" + 0.007*"chicken" + 0.007*"chocolate" + 0.006*"cream" + 0.006*"salad

In [75]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model_tfidf, corpus=bow, texts=preproc_doc)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,23.0,0.4821,"blog, post, urllink, link, blogger, comment, s...","[thanks, yahoo, toolbar, capture, url, popups,..."
1,1,15.0,0.2839,"urllink, bush, iraq, people, american, preside...","[interesting, conversation, morning, talking, ..."
2,2,19.0,0.6232,"life, know, love, want, thing, like, feel, tim...","[coca, cola, summing, thing, early, flagship, ..."
3,3,15.0,0.4744,"urllink, bush, iraq, people, american, preside...","[korea, country, extreme, based, think, come, ..."
4,4,15.0,0.7757,"urllink, bush, iraq, people, american, preside...","[read, news, article, urllink, joongang, ilbo,..."
5,5,15.0,0.5434,"urllink, bush, iraq, people, american, preside...","[surf, english, news, site, looking, tidbit, k..."
6,6,2.0,0.2729,"like, time, night, little, went, thing, good, ...","[korean, language, look, difficult, figure, re..."
7,7,2.0,0.3697,"like, time, night, little, went, thing, good, ...","[click, profile, startling, discovery, born, y..."
8,8,9.0,0.5489,"went, work, going, today, time, like, night, w...","[night, pretty, company, kept, recently, coupl..."
9,9,7.0,0.6604,"today, like, went, good, going, night, urllink...","[different, seen, haven, travelled, canada, ph..."


In [76]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_tfidf, bow, dictionary=lda_model_tfidf.id2word)
vis

C:\Users\jinit\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
19    -0.158111  0.006642       1        1  24.158083
9     -0.139495 -0.042467       2        1  15.643212
2     -0.130852 -0.009809       3        1  14.560500
15    -0.120003  0.152324       4        1  12.739565
17    -0.111702  0.150230       5        1   5.119747
13    -0.100808 -0.023578       6        1   3.088969
11    -0.094218 -0.043725       7        1   2.862975
23    -0.065884  0.073759       8        1   2.608728
29    -0.067277  0.127382       9        1   2.587978
10    -0.060739  0.009403      10        1   2.413044
27     0.006512 -0.093692      11        1   2.235267
7     -0.091261 -0.073850      12        1   1.721245
28    -0.078300  0.021696      13        1   1.573785
5      0.016993 -0.157339      14        1   1.159318
0     -0.076105 -0.077366      15        1   0.922820
20    -0.047660 -0.048773      16        1   0.910524
26    -0.023866 -0.080342      17        1   0.834704
14    -0.014539  0.079870      18        1   0.811901
25    -0.026060 -0.034522      19        1   0.654373
18     0.045961 -0.005971      20        1   0.402019
4      0.029952 -0.074654      21        1   0.399043
12     0.058252  0.073032      22        1   0.353136
24     0.166669  0.057928      23        1   0.310659
22     0.103226 -0.087589      24        1   0.310512
3      0.123356  0.039082      25        1   0.309983
21     0.213379  0.075136      26        1   0.283326
1      0.137186 -0.007496      27        1   0.278531
8      0.120639 -0.050298      28        1   0.266485
6      0.229405  0.022664      29        1   0.253142
16     0.155351  0.022326      30        1   0.226418, topic_info=      Category           Freq       Term          Total  loglift  logprob
term                                                                     
2288   Default  145924.000000       nbsp  145924.000000  30.0000  30.0000
21     Default  120576.000000    urllink  120576.000000  29.0000  29.0000
575    Default   39276.000000       blog   39276.000000  28.0000  28.0000
200    Default   29924.000000       post   29924.000000  27.0000  27.0000
88     Default   61631.000000       know   61631.000000  26.0000  26.0000
10     Default   63401.000000       like   63401.000000  25.0000  25.0000
694    Default   43295.000000      today   43295.000000  24.0000  24.0000
728    Default   48816.000000       love   48816.000000  23.0000  23.0000
2110   Default   24370.000000      happy   24370.000000  22.0000  22.0000
4419   Default   13298.000000       http   13298.000000  21.0000  21.0000
67     Default   48525.000000       good   48525.000000  20.0000  20.0000
219    Default   51384.000000      think   51384.000000  19.0000  19.0000
224    Default   46276.000000       work   46276.000000  18.0000  18.0000
65     Default   47016.000000      going   47016.000000  17.0000  17.0000
314    Default   58292.000000       time   58292.000000  16.0000  16.0000
161    Default   53550.000000      thing   53550.000000  15.0000  15.0000
16     Default   19442.000000       site   19442.000000  14.0000  14.0000
3436   Default   19852.000000       song   19852.000000  13.0000  13.0000
560    Default   49935.000000       want   49935.000000  12.0000  12.0000
3120   Default   13674.000000   birthday   13674.000000  11.0000  11.0000
564    Default   35767.000000       week   35767.000000  10.0000  10.0000
1050   Default   48415.000000       life   48415.000000   9.0000   9.0000
466    Default   22002.000000    picture   22002.000000   8.0000   8.0000
692    Default   12685.000000       test   12685.000000   7.0000   7.0000
484    Default   33542.000000       went   33542.000000   6.0000   6.0000
463    Default   36955.000000      night   36955.000000   5.0000   5.0000
4      Default    8372.000000      click    8372.000000   4.0000   4.0000
11     Default   16039.000000       link   16039.000000   3.0000   3.0000
196    De